## Security Functions Test

Day1 작업 중에 `src/app/core/security.py` 관련 함수 테스트

#### Test Coverage
- `create_magic_link_token()`: Magic link token generation
- `create_access_token()`: Access token generation
- `verify_token()`: Token verification with type checking

In [1]:
import sys
from pathlib import Path

# Add src directory to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

print(f"Project root: {project_root}")
print(f"Python path: {sys.path[0]}")

Project root: /mnt/d/project/research-curator
Python path: /mnt/d/project/research-curator/src


In [2]:
from datetime import UTC, datetime, timedelta
from jose import jwt
from app.core.security import create_magic_link_token, create_access_token, verify_token
from app.core.config import settings

print("✅ All imports successful")
print(f"JWT Algorithm: {settings.JWT_ALGORITHM}")
print(f"Magic Link Expiry: {settings.MAGIC_LINK_EXPIRE_MINUTES} minutes")
print(f"Access Token Expiry: {settings.ACCESS_TOKEN_EXPIRE_DAYS} days")

✅ All imports successful
JWT Algorithm: HS256
Magic Link Expiry: 15 minutes
Access Token Expiry: 30 days


### 1. 매직 링크 토큰 생성(Magic Link Token Creation)

In [3]:
# Test magic link token creation
test_email = "test@example.com"
magic_token = create_magic_link_token(test_email)

print(f"Magic link token created: {magic_token[:50]}...")
print(f"Token length: {len(magic_token)} characters")

Magic link token created: eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJzdWIiOiJ0Z...
Token length: 165 characters


In [4]:
# Decode to inspect payload
payload = jwt.decode(magic_token, settings.JWT_SECRET_KEY, algorithms=[settings.JWT_ALGORITHM])
print(f"\nToken payload:")
print(f"  - Email (sub): {payload.get('sub')}")
print(f"  - Type: {payload.get('type')}")
print(f"  - Expiry: {datetime.fromtimestamp(payload.get('exp'), UTC)}")


Token payload:
  - Email (sub): test@example.com
  - Type: magic_link
  - Expiry: 2025-12-12 03:33:32+00:00


In [5]:
# Verify token type and email
assert payload.get('sub') == test_email, "Email mismatch"
assert payload.get('type') == 'magic_link', "Token type should be magic_link"
print("\n✅ Magic link token creation test passed")


✅ Magic link token creation test passed


### 2. 액세스 토큰 생성(Access Token Creation)

In [6]:
# Test access token creation
test_email = "user@example.com"
access_token = create_access_token(test_email)

print(f"Access token created: {access_token[:50]}...")
print(f"Token length: {len(access_token)} characters")

Access token created: eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJzdWIiOiJ1c...
Token length: 160 characters


In [7]:
# Decode to inspect payload
payload = jwt.decode(access_token, settings.JWT_SECRET_KEY, algorithms=[settings.JWT_ALGORITHM])
print(f"\nToken payload:")
print(f"  - Email (sub): {payload.get('sub')}")
print(f"  - Type: {payload.get('type')}")
print(f"  - Expiry: {datetime.fromtimestamp(payload.get('exp'), UTC)}")


Token payload:
  - Email (sub): user@example.com
  - Type: access
  - Expiry: 2026-01-11 03:18:32+00:00


In [8]:
# Verify token type and email
assert payload.get('sub') == test_email, "Email mismatch"
assert payload.get('type') == 'access', "Token type should be access"
print("\n✅ Access token creation test passed")


✅ Access token creation test passed


### 3. 토큰 검증(Token Verification) - Valid Tokens

In [9]:
# Test verifying valid magic link token
test_email = "magic@example.com"
magic_token = create_magic_link_token(test_email)
verified_email = verify_token(magic_token, expected_type="magic_link")

print(f"Original email: {test_email}")
print(f"Verified email: {verified_email}")
assert verified_email == test_email, "Email should match"
print("✅ Magic link verification passed\n")

Original email: magic@example.com
Verified email: magic@example.com
✅ Magic link verification passed



In [10]:
# Test verifying valid access token
test_email = "access@example.com"
access_token = create_access_token(test_email)
verified_email = verify_token(access_token, expected_type="access")

print(f"Original email: {test_email}")
print(f"Verified email: {verified_email}")
assert verified_email == test_email, "Email should match"
print("✅ Access token verification passed")

Original email: access@example.com
Verified email: access@example.com
✅ Access token verification passed


### 4. 토큰 검증(Token Verification) - Type Mismatch

In [11]:
# Test type mismatch: magic_link token verified as access
magic_token = create_magic_link_token("test@example.com")
result = verify_token(magic_token, expected_type="access")

print(f"Magic token verified as access: {result}")
assert result is None, "Should return None for type mismatch"
print("✅ Type mismatch test passed (magic -> access)\n")

Magic token verified as access: None
✅ Type mismatch test passed (magic -> access)



In [12]:
# Test type mismatch: access token verified as magic_link
access_token = create_access_token("test@example.com")
result = verify_token(access_token, expected_type="magic_link")

print(f"Access token verified as magic_link: {result}")
assert result is None, "Should return None for type mismatch"
print("✅ Type mismatch test passed (access -> magic_link)")

Access token verified as magic_link: None
✅ Type mismatch test passed (access -> magic_link)


### 5. 토큰 검증(Token Verification) - Invalid Tokens

In [13]:
# Test invalid token format
invalid_token = "invalid.token.format"
result = verify_token(invalid_token)

print(f"Invalid token result: {result}")
assert result is None, "Should return None for invalid token"
print("✅ Invalid token format test passed\n")

Invalid token result: None
✅ Invalid token format test passed



In [14]:
# Test completely malformed token
malformed_token = "not-even-a-jwt"
result = verify_token(malformed_token)

print(f"Malformed token result: {result}")
assert result is None, "Should return None for malformed token"
print("✅ Malformed token test passed")

Malformed token result: None
✅ Malformed token test passed


### 6. 토큰 검증(Token Verification) - Expired Token

In [15]:
# Create an expired token (expired 1 minute ago)
test_email = "expired@example.com"
expire = datetime.now(UTC) - timedelta(minutes=1)
payload = {
    "sub": test_email,
    "exp": expire,
    "type": "access",
}
expired_token = jwt.encode(payload, settings.JWT_SECRET_KEY, algorithm=settings.JWT_ALGORITHM)

print(f"Expired token created (expired at: {expire})")
result = verify_token(expired_token)

print(f"Verification result: {result}")
assert result is None, "Should return None for expired token"
print("✅ Expired token test passed")

Expired token created (expired at: 2025-12-12 03:17:33.026339+00:00)
Verification result: None
✅ Expired token test passed


### 7. 토큰 검증(Token Verification) - Missing Fields, 필수 필드 누락

In [16]:
# Test token without 'sub' field
expire = datetime.now(UTC) + timedelta(days=1)
payload = {
    "exp": expire,
    "type": "access",
}
no_sub_token = jwt.encode(payload, settings.JWT_SECRET_KEY, algorithm=settings.JWT_ALGORITHM)
result = verify_token(no_sub_token)

print(f"Token without 'sub' result: {result}")
assert result is None, "Should return None when 'sub' is missing"
print("✅ Missing 'sub' field test passed\n")

Token without 'sub' result: None
✅ Missing 'sub' field test passed



In [17]:
# Test token without 'type' field
payload = {
    "sub": "test@example.com",
    "exp": expire,
}
no_type_token = jwt.encode(payload, settings.JWT_SECRET_KEY, algorithm=settings.JWT_ALGORITHM)
result = verify_token(no_type_token)

print(f"Token without 'type' result: {result}")
assert result is None, "Should return None when 'type' is missing"
print("✅ Missing 'type' field test passed")

Token without 'type' result: None
✅ Missing 'type' field test passed


### 8. 만료 시간 검증(Token Expiration Times)

In [18]:
# Test magic link token expiration time
magic_token = create_magic_link_token("test@example.com")
payload = jwt.decode(magic_token, settings.JWT_SECRET_KEY, algorithms=[settings.JWT_ALGORITHM])
exp_time = datetime.fromtimestamp(payload.get('exp'), UTC)
time_diff = exp_time - datetime.now(UTC)

print(f"Magic link token expiration:")
print(f"  - Expires at: {exp_time}")
print(f"  - Time remaining: {time_diff}")
print(f"  - Expected: {settings.MAGIC_LINK_EXPIRE_MINUTES} minutes")

# Allow 1 second tolerance
expected_seconds = settings.MAGIC_LINK_EXPIRE_MINUTES * 60
assert abs(time_diff.total_seconds() - expected_seconds) < 1, "Expiration time mismatch"
print("✅ Magic link expiration time correct\n")

# Test access token expiration time
access_token = create_access_token("test@example.com")
payload = jwt.decode(access_token, settings.JWT_SECRET_KEY, algorithms=[settings.JWT_ALGORITHM])
exp_time = datetime.fromtimestamp(payload.get('exp'), UTC)
time_diff = exp_time - datetime.now(UTC)

print(f"Access token expiration:")
print(f"  - Expires at: {exp_time}")
print(f"  - Time remaining: {time_diff}")
print(f"  - Expected: {settings.ACCESS_TOKEN_EXPIRE_DAYS} days")

expected_seconds = settings.ACCESS_TOKEN_EXPIRE_DAYS * 24 * 60 * 60
assert abs(time_diff.total_seconds() - expected_seconds) < 1, "Expiration time mismatch"
print("✅ Access token expiration time correct")

Magic link token expiration:
  - Expires at: 2025-12-12 03:33:33+00:00
  - Time remaining: 0:14:59.949423
  - Expected: 15 minutes
✅ Magic link expiration time correct

Access token expiration:
  - Expires at: 2026-01-11 03:18:33+00:00
  - Time remaining: 29 days, 23:59:59.949052
  - Expected: 30 days
✅ Access token expiration time correct


### Test Summary

In [19]:
print("="*50)
print("All Security Tests Passed! ✅")
print("="*50)
print("\nTest Coverage:")
print("  ✅ Magic link token creation")
print("  ✅ Access token creation")
print("  ✅ Valid token verification")
print("  ✅ Token type mismatch handling")
print("  ✅ Invalid token format handling")
print("  ✅ Expired token handling")
print("  ✅ Missing field handling")
print("  ✅ Expiration time validation")

All Security Tests Passed! ✅

Test Coverage:
  ✅ Magic link token creation
  ✅ Access token creation
  ✅ Valid token verification
  ✅ Token type mismatch handling
  ✅ Invalid token format handling
  ✅ Expired token handling
  ✅ Missing field handling
  ✅ Expiration time validation
